SQLite3
- sqlite3.connect(filename.db)
- conn.cursor()
- cursor.execute(SQLStatement, object) 

cursor.execute(SQLStatement, object), object is optional.

In [1]:
import sqlite3

conn = sqlite3.connect("datafile.db")
cursor = conn.cursor()
cursor.execute("""
               create table people (id integer primary key, name text, count integer)""")
cursor.execute("""insert into people (name, count) values ('Sandy', 25)""")

conn.commit()
conn.close()

In [6]:
import sqlite3

conn = sqlite3.connect("datafile.db")
cursor = conn.cursor()
cursor.execute('insert into people (name, count) values (?, ?)', ('Bob', 25))
cursor.execute("""insert into people (name, count) values (:username, :usercount)""", {"username": "Grace", "usercount": 27})

conn.commit()
conn.close()

sqlite3
- result.fetchall(), 提取全部的東西，returns a list of tuples, each row of data in the database in stored in one tuple.
- result.fetchmany(n), return a list of the first nth tuples
- result.fetchone(), return a tuple of the first match for the SQL statement

In [17]:
import sqlite3

conn = sqlite3.connect("datafile.db")
cursor = conn.cursor()

result = cursor.execute("select * from people")

print(result.fetchall())
# print(result.fetchone())
# print(result.fetchmany(2))

conn.commit()
conn.close()


[(1, 'Sandy', 25), (2, 'Bob', 25), (3, 'Grace', 27)]


sqlite3
- conn.commit(), save the changes to the database
- conn.close(), close the database connection
- conn.rollback(), 實際應用上不會用到，會用其他工具如 Alembic 去控制不同版本的資料庫

CRUD
- create new data rows in the database (insert into)
- read data rows from the database (select * from <table name> where ...)
- update existing rows in the database
- delete certain rows in the database

In [ ]:
import sqlite3

conn = sqlite3.connect("datafile.db")
cursor = conn.cursor()

#cursor.execute('update people set count = :usercount where name = :username', {"username": 'Bob', "usercount": 39})
cursor.execute("""Delete from people where name='Grace'""")

result = cursor.execute("select * from people")
print(result.fetchall())

conn.commit()
conn.close()

OperationalError: database is locked

SQL injection

攻擊者藉由在輸入欄位或 API 請求中注入惡意 SQL 語句，讓系統執行攻擊者對指定資料庫進行以下操作:
- 竊取資料
- 繞過驗證
- 修改或刪除資料
- 執行管理命令
- 控制伺服器或提升權限

參數化可以避免這個攻擊
parameterized

result = cursor.execute("""select * from people where name = :username""", {"username": name})
result = cursor.execute("""select * from people where name = ?""", (name,))

sql Alchemy, Django ORM
Handle Database Easier with an ORM
- Object Relational Mapping 物件關聯對應
  解決物件導向程式(java, python)與關聯式資料庫(Mysql, Postgres)之間的阻抗不匹配，
  將資料庫中的資料以物件方式表示，讓開發者能夠以物件導向的語法來操作資料庫，
  從而提升開發效率、程式碼的可維護性與可讀性，並支援跨資料庫的可疑質性且預防 sql 注入攻擊。

In [20]:
pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 991.0 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 1.6 MB/s eta 0:00:02
   ------- -------------------------------- 0.4/2.1 MB 3.6 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.1 MB 6.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.1 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/299.9 kB ? eta -:--:--
   --------------------------------------- 299.9/299.9 kB 18.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import sqlite3
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String 
from sqlalchemy.orm import sessionmaker

dbPath = 'datafile.db'
engine = create_engine('sqlite:///%s' % dbPath)
metadata = MetaData(engine)
people = Table('people', metadata, Column('id', Integer, primary_key=True), Column('name', String), Column('count', Integer))
Session = sessionmaker(bind=engine)
session = Session()
metadata.create_all(engine)

insert_statement = people.insert().values(name='Spencer', count=66)
session.execute(insert_statement)
session.commit()

result = session.execute(select([people]))
for row in result:
    print(row)

ArgumentError: expected schema argument to be a string, got <class 'sqlalchemy.engine.base.Engine'>.